In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import time
from resume_parser import resumeparse
import cohere
import numpy as np
from annoy import AnnoyIndex
import umap.umap_ as umap
import altair as alt
import pyrebase

In [8]:
ls_final = []
data = resumeparse.read_file(r'C:\Users\joshu\job-basket\Tim Resume.pdf')
ls_jobs = data['designition']
for i in ls_jobs:
    if(len(i.split()) == 2):
        ls_final.append(i)
ls_final

['learning engineer', 'data scientist']

In [9]:
for val in ls_final:
    print(val.split()[0])
    print(val.split()[1])

learning
engineer
data
scientist


In [10]:
def getInput():
    keywords=[]
    while (True):
        word = input("Enter a keyword. Enter \"done\" to finish.")
        if word.lower()=="done":
            break
        keywords.append(word)
    jt = input("full time, permanent, contract, apprenticeship, intern, part-time, temporary, casual")
    searchByKeyword(keywords, jt)

In [11]:
def searchByKeyword(title1, title2, i, jobList):
    search = "https://ca.indeed.com/jobs?q=" + title1 + "%20" + title2 + "&start=" + str(i)

    session = requests.Session()
    response = session.get(search, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})
    soup = BeautifulSoup(response.text, 'html.parser')
    print(search)

    # array of dictionary

    jobList = extract_info(soup)
    links = extract_link(soup)
    
    for i in range(0, len(links)):
        job_src = requests.get(links[i]).text
        job_soup = BeautifulSoup(job_src, 'lxml')
        salary, job_type, desc = extract_details(job_soup)
        jobList[i]['link']=links[i]
        jobList[i]['salary']=salary
        jobList[i]['job_type']=job_type
        jobList[i]['description']=desc
    return jobList

def extract_link(soup): 
    links = []
    part = soup.find('div', id='mosaic-provider-jobcards')
    for elem in part.find_all('a', class_='tapItem'):
        link = elem.get('href')
        link="https://ca.indeed.com"+link
        links.append(link)
    return(links)

def extract_info(soup):
    jobs = []
    job_titles = []
    company_names = []
    locations = []
    for elem in soup.find_all('h2', class_='jobTitle'):
        spans = elem.find_all('span')
        for s in spans:
            if s.has_attr('title'):
                title = s.get_text()
        job_titles.append(title)
        
    for elem in soup.find_all('span', class_='companyName'):
        name = elem.get_text()
        company_names.append(name)
        
    for elem in soup.find_all('div', class_='companyLocation'):
        loc = elem.get_text()
        if loc.find(" in ")!=-1:
            idx = loc.index(" in ")+4
            loc = loc[idx:]
        locations.append(loc)
        
    for a in range(0, len(job_titles)):
        dic = {}
        dic['title']=job_titles[a]
        dic['company']=company_names[a]
        dic['location']=locations[a]
        jobs.append(dic)
        
    return jobs
    
def extract_details(soup):
    container = soup.find('div', id='salaryInfoAndJobType')
    desc_cont = soup.find('div', id='jobDescriptionText')
    if desc_cont:
        description = desc_cont.get_text()
    salary=""
    job_type=""
    if container:
        salary_cont = container.find("span", recursive=False, class_='attribute_snippet')
        job_type_cont = container.find("span", recursive=False, class_='jobsearch-JobMetadataHeader-item')
        if salary_cont:
            salary = salary_cont.get_text()
        if job_type_cont:
            job_type = job_type_cont.get_text()
    return salary, job_type, description

final_list = []

for val in ls_final:
    for i in range(0, 30, 10):
        jobList= []
        jobList = searchByKeyword(val.split()[0], val.split()[1], i, jobList)
        final_list.append(jobList)

https://ca.indeed.com/jobs?q=learning%20engineer&start=0
https://ca.indeed.com/jobs?q=learning%20engineer&start=10
https://ca.indeed.com/jobs?q=learning%20engineer&start=20
https://ca.indeed.com/jobs?q=data%20scientist&start=0
https://ca.indeed.com/jobs?q=data%20scientist&start=10
https://ca.indeed.com/jobs?q=data%20scientist&start=20


In [19]:
final_list

[[{'title': 'Application Engineer',
   'company': 'EXO Insights',
   'location': 'Waterloo, ON',
   'link': 'https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bie-wKyw0HPz5nXk6_cEGdbviBINuiUd9XyZ0s5xPPbozfQRRArtELqTrd5TXX3QVAzzTG9Ynh456VxXWZ--dj7LfhlAwUtxz6g9o5N2I0QlO2CUqJxqxhetzPIff0n-bVAMnWugo5RS2wjvCwbApY_0vhde7cviAQFI75af4GxZKVXhY_xGFcqO_1SvEFsEphLs4tzfiFgitSChVw2LDVGs1wHnEd5I_HStkEfXq8DVJpRbHPkE-G2KsKWPKQ2WZpiNSwLnzK64P-VFNaDVTmoOgOa7u3jQJ4ZYLBEuyck-oJU-P-oYsH0g0o_JNwuWx3q_8HBmck0Cf37beWEO2AjGtpGnBvcW3iVMIo1uZI-TxyJNxpkE-89Qa8ygz2of74i5X6GyDhWkdCBb21xPzm88MqgmpjZwwcSKmjiry7TEzb9IWCzjt-Z7kAophUvUt2BdhlveJGtnwdZSTSL0iHVmuQNp2y-cNrfyNK9Gq3Bw==&p=0&fvj=1&vjs=3',
   'salary': '$50,000 - $65,000 a year',
   'job_type': ' -  Full-time, Permanent',
   'description': 'About the JobEXO Insights is seeking an Application Engineer to assist in advancing our technical capabilities, and to assist in keeping EXO at the technological frontier. With EXO we are constantly learning new skills, disc

In [30]:
dataTitle = []
dataCompany = []
dataSalary = []         
dataLocation = []
dataDescription = []
dataType = []
datalink = []
counter = 0
for val in final_list:
    for element in val:
        if(counter > 13):
            break
        dataTitle.append(element['title'])
        dataCompany.append(element['company'])
        dataSalary.append(element['salary'])
        dataType.append(element['job_type'])
        dataLocation.append(element['location'])
        dataDescription.append(element['description'])
        datalink.append(element['link'])
        counter += 1

df = pd.DataFrame({'title': dataTitle, 'company': dataCompany, 'salary': dataSalary, 'location': dataLocation, 'description': dataDescription, 'job type': dataType, 'link': datalink})
df = df.iloc[1: , :]

In [31]:
df

,title,company,salary,location,description,job type,link
1,Machine Learning Engineer,Planitar,"$65,851 - $109,740 a year","Waterloo, ON",POSITION SUMMARYThe successful candidate will ...,"- Full-time, Permanent",https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Associate Project Engineer,ABB,-1,"Burlington, ON",Associate Project Engineer\nTake your next car...,Full-time,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,Quality Engineer,Manulife,-1,"Waterloo, ON","Are you looking for a supportive, collaborativ...",Full-time,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,Jr. 2LS Support Engineer,NOKIA,-1,"Ottawa, ON",Come create the technology that helps the worl...,,https://ca.indeed.com/rc/clk?jk=bd0b8d0b0e36e3...
5,Database Engineer,Royal Bank of Canada,-1,"Toronto, ON","Come Work with Us!\n\nAt RBC, our culture is d...",Full-time,https://ca.indeed.com/rc/clk?jk=2ed6b6e5e57c1d...
6,Machine Learning Engineer,nugget.ai,-1,"Toronto, ON",nugget.ai is working in partnership with Plane...,"Full-time, Permanent",https://ca.indeed.com/rc/clk?jk=8c9ada1c6620f4...
7,Product Development Engineer,Leggett & Platt,-1,"Waterloo, ON","We, at Leggett & Platt Inc., are searching for...",,https://ca.indeed.com/rc/clk?jk=0e525e1c656343...
8,Junior Climate Resilience Professional,Wood Plc,-1,"Oakville, ON",Overview / Responsibilities:\n\nWood is curren...,,https://ca.indeed.com/rc/clk?jk=69ece992a88c2e...
9,Frontend Engineer,AgencyAnalytics,"$80,000 - $110,000 a year",Remote,Our Experience team is looking for a passionat...,- Full-time,https://ca.indeed.com/rc/clk?jk=3a86fea009ba50...
10,Projects Engineer,Agnico Eagle Mines Limited,-1,"Cochrane, ON",If you thrive on success and team work in a te...,Full-time,https://ca.indeed.com/company/Kirkland-Lake-Go...


In [32]:
string = ' '.join(data['skills'])
string

': Python · SQL · Spark · Bash · C/C++ · Javascript · HTML/CSS Libraries: Pandas · Scikitlearn · NumPy · Matplotlib · PyTorch · Keras · OpenCV · AutoML Tools / Techniques: Azure DevOps · PowerBI · Databricks · Jupyter · Excel · Machine Learning · Deep Learning'

In [33]:
co = cohere.Client('AxgVH1dlcPmLWWvfujVYLcZTta67uZUnTHp92NKB')

In [34]:
embeds = co.embed(texts=list(df['description']), model="large", truncate="LEFT").embeddings

In [35]:
embeds = np.array(embeds)
embeds.shape

(13, 4096)

In [36]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees 
search_index.save('test.ann')

True

In [43]:
query = string

# Get the query's embedding
query_embed = co.embed(texts=[query],
                  model="large",
                  truncate="LEFT").embeddings

# Retrieve the nearest neighbors
similar_item_ids = search_index.get_nns_by_vector(query_embed[0],20,
                                                include_distances=True)
# Format the results
results = pd.DataFrame(data={'links': df.iloc[similar_item_ids[0]]['link'], 
                             'distance': similar_item_ids[1]})
print(f"Query:'{query}'\nNearest neighbors:")
results

Query:': Python · SQL · Spark · Bash · C/C++ · Javascript · HTML/CSS Libraries: Pandas · Scikitlearn · NumPy · Matplotlib · PyTorch · Keras · OpenCV · AutoML Tools / Techniques: Azure DevOps · PowerBI · Databricks · Jupyter · Excel · Machine Learning · Deep Learning'
Nearest neighbors:


,links,distance
1,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...,0.860067
6,https://ca.indeed.com/rc/clk?jk=8c9ada1c6620f4...,0.886785
11,https://ca.indeed.com/rc/clk?jk=94c3c8a01cef34...,0.933849
13,https://ca.indeed.com/rc/clk?jk=9f7b5aff42ce9d...,1.089439
5,https://ca.indeed.com/rc/clk?jk=2ed6b6e5e57c1d...,1.098259
9,https://ca.indeed.com/rc/clk?jk=3a86fea009ba50...,1.100359
4,https://ca.indeed.com/rc/clk?jk=bd0b8d0b0e36e3...,1.205194
3,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...,1.232202
12,https://ca.indeed.com/company/DEI-Consulting-E...,1.235236
2,https://ca.indeed.com/pagead/clk?mo=r&ad=-6NYl...,1.243945


In [44]:
results.to_json(r"C:\Users\joshu\job-basket\app\src\dataframe.json", orient = "columns")

In [45]:
results['links'].iloc[4]

'https://ca.indeed.com/rc/clk?jk=2ed6b6e5e57c1dad&fccid=537b899e30af3338&vjs=3'